# Data Analytics Assignment 2024

In [1]:
##2. Reading and Cleaning Data
##Use Pandas for reading and cleaning data.
import pandas as pd

# Reading data
stdData = pd.read_csv("StudentPerformanceFactors.csv")
stdData


,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
0,23,84,Low,High,No,7,73,Low,Yes,0,Low,Medium,Public,Positive,3,No,High School,Near,Male,67
1,19,64,Low,Medium,No,8,59,Low,Yes,2,Medium,Medium,Public,Negative,4,No,College,Moderate,Female,61
2,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,Medium,Medium,Public,Neutral,4,No,Postgraduate,Near,Male,74
3,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,Medium,Medium,Public,Negative,4,No,High School,Moderate,Male,71
4,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,Medium,High,Public,Neutral,4,No,College,Near,Female,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6602,25,69,High,Medium,No,7,76,Medium,Yes,1,High,Medium,Public,Positive,2,No,High School,Near,Female,68
6603,23,76,High,Medium,No,8,81,Medium,Yes,3,Low,High,Public,Positive,2,No,High School,Near,Female,69
6604,20,90,Medium,Low,Yes,6,65,Low,Yes,3,Low,Medium,Public,Negative,2,No,Postgraduate,Near,Female,68
6605,10,86,High,High,Yes,6,91,High,Yes,2,Low,Medium,Private,Positive,3,No,High School,Far,Female,68


In [2]:
stdData.describe()

,Hours_Studied,Attendance,Sleep_Hours,Previous_Scores,Tutoring_Sessions,Physical_Activity,Exam_Score
count,6607.000000,6607.000000,6607.00000,6607.000000,6607.000000,6607.000000,6607.000000
mean,19.975329,79.977448,7.02906,75.070531,1.493719,2.967610,67.235659
std,5.990594,11.547475,1.46812,14.399784,1.230570,1.031231,3.890456
min,1.000000,60.000000,4.00000,50.000000,0.000000,0.000000,55.000000
25%,16.000000,70.000000,6.00000,63.000000,1.000000,2.000000,65.000000
50%,20.000000,80.000000,7.00000,75.000000,1.000000,3.000000,67.000000
75%,24.000000,90.000000,8.00000,88.000000,2.000000,4.000000,69.000000
max,44.000000,100.000000,10.00000,100.000000,8.000000,6.000000,101.000000


In [3]:
stdData.head()

,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
0,23,84,Low,High,No,7,73,Low,Yes,0,Low,Medium,Public,Positive,3,No,High School,Near,Male,67
1,19,64,Low,Medium,No,8,59,Low,Yes,2,Medium,Medium,Public,Negative,4,No,College,Moderate,Female,61
2,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,Medium,Medium,Public,Neutral,4,No,Postgraduate,Near,Male,74
3,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,Medium,Medium,Public,Negative,4,No,High School,Moderate,Male,71
4,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,Medium,High,Public,Neutral,4,No,College,Near,Female,70


In [4]:
stdData.shape

(6607, 20)

In [5]:
stdData.columns

Index(['Hours_Studied', 'Attendance', 'Parental_Involvement',
       'Access_to_Resources', 'Extracurricular_Activities', 'Sleep_Hours',
       'Previous_Scores', 'Motivation_Level', 'Internet_Access',
       'Tutoring_Sessions', 'Family_Income', 'Teacher_Quality', 'School_Type',
       'Peer_Influence', 'Physical_Activity', 'Learning_Disabilities',
       'Parental_Education_Level', 'Distance_from_Home', 'Gender',
       'Exam_Score'],
      dtype='object')

In [6]:
#Deletion of unnecessary columns
stdData = stdData.drop(columns=['Extracurricular_Activities', 'Previous_Scores', 'Motivation_Level', 'Tutoring_Sessions', 'Peer_Influence', 'Physical_Activity', 'Learning_Disabilities', 'Parental_Education_Level', 'Distance_from_Home', 'Gender'])

In [7]:
stdData.head

<bound method NDFrame.head of       Hours_Studied  Attendance Parental_Involvement Access_to_Resources  \
0                23          84                  Low                High   
1                19          64                  Low              Medium   
2                24          98               Medium              Medium   
3                29          89                  Low              Medium   
4                19          92               Medium              Medium   
...             ...         ...                  ...                 ...   
6602             25          69                 High              Medium   
6603             23          76                 High              Medium   
6604             20          90               Medium                 Low   
6605             10          86                 High                High   
6606             15          67               Medium                 Low   

      Sleep_Hours Internet_Access Family_Income Teacher_Q

In [8]:
stdData.head(10)

,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Sleep_Hours,Internet_Access,Family_Income,Teacher_Quality,School_Type,Exam_Score
0,23,84,Low,High,7,Yes,Low,Medium,Public,67
1,19,64,Low,Medium,8,Yes,Medium,Medium,Public,61
2,24,98,Medium,Medium,7,Yes,Medium,Medium,Public,74
3,29,89,Low,Medium,8,Yes,Medium,Medium,Public,71
4,19,92,Medium,Medium,6,Yes,Medium,High,Public,70
5,19,88,Medium,Medium,8,Yes,Medium,Medium,Public,71
6,29,84,Medium,Low,7,Yes,Low,Medium,Private,67
7,25,78,Low,High,6,Yes,High,High,Public,66
8,17,94,Medium,High,6,Yes,Medium,Low,Private,69
9,23,98,Medium,Medium,8,Yes,High,High,Public,72


In [9]:
stdData.columns.value_counts()

Hours_Studied           1
Attendance              1
Parental_Involvement    1
Access_to_Resources     1
Sleep_Hours             1
Internet_Access         1
Family_Income           1
Teacher_Quality         1
School_Type             1
Exam_Score              1
Name: count, dtype: int64

In [10]:
# checks for missing (NaN) values in the stdData DataFrame and returns a count of how many missing values are in each column.
missing_values = stdData.isnull().sum()
missing_values

Hours_Studied            0
Attendance               0
Parental_Involvement     0
Access_to_Resources      0
Sleep_Hours              0
Internet_Access          0
Family_Income            0
Teacher_Quality         78
School_Type              0
Exam_Score               0
dtype: int64

In [11]:
missing_values = stdData.isna().sum()
missing_values

Hours_Studied            0
Attendance               0
Parental_Involvement     0
Access_to_Resources      0
Sleep_Hours              0
Internet_Access          0
Family_Income            0
Teacher_Quality         78
School_Type              0
Exam_Score               0
dtype: int64

In [15]:
# delete all the rows with missing values
# stdData_cleaned = stdData.dropna()
# agreed not to drop

In [ ]:
# # Extract the 'Teacher_Quality' column
# df_teacher_quality = df['Teacher_Quality']

# # Drop the 'Teacher_Quality' column from the DataFrame
# df_new = df.drop('Teacher_Quality', axis=1)

# # Replace missing values in the remaining columns with their respective modes
# for column in df_new.columns:
#     mode_value = df_new[column].mode()[0]  # Get the mode for the column
#     df_new[column].fillna(mode_value, inplace=True)  # Replace NaNs with the mode

# # Now replace missing values in the 'Teacher_Quality' column with its mode
# mode_teacher_quality = df_teacher_quality.mode()[0]
# df_teacher_quality.fillna(mode_teacher_quality, inplace=True)

# # Optionally, reinsert the 'Teacher_Quality' column back into df_new
# df_new['Teacher_Quality'] = df_teacher_quality

# # Display the new DataFrame to verify
# print(df_new)